In this notebook we simulate rounded cubes using the [DEM component](https://hoomd-blue.readthedocs.io/en/stable/module-dem-pair.html#hoomd.dem.pair.WCA) of HOOMD-Blue. We visualize the shapes using the povray backend and color them by the distance to their nearest neighbor.

In [ ]:
import flowws
import gtar
from hoomd_flowws.Init import Init
from hoomd_flowws.DEMInteraction import DEMInteraction
from hoomd_flowws.Run import Run
from hoomd_flowws.ShapeDefinition import ShapeDefinition

import json
import plato, plato.draw.povray as draw
from plato.draw.vispy.internal import DEFAULT_DIRECTIONAL_LIGHTS as directional_lights
import freud
import numpy as np
import IPython

In [ ]:
shape = dict(type='cube', modifications=[dict(type='scale', factor=2), dict(type='round', radius=.5)])

storage = flowws.DirectoryStorage()
stages = [
    ShapeDefinition(shape_arguments=[shape]),
    Init(number=128),
    DEMInteraction(type='wca'),
    Run(steps=1e3, integrator='langevin'),
    Run(steps=1e5, integrator='langevin', compress_to=.8, dump_period=1e4),
]

flowws.Workflow(stages, storage).run()

In [ ]:
with gtar.GTAR('dump.sqlite', 'r') as traj:
    (posRec, quatRec, boxRec), frames = traj.framesWithRecordsNamed(['position', 'orientation', 'box'])
    
    for (_, type_shapes) in traj.recordsNamed('type_shapes.json'):
        type_shapes = json.loads(type_shapes)
    
    positions = traj.getRecord(posRec, frames[-1])
    orientations = traj.getRecord(quatRec, frames[-1])
    box = traj.getRecord(boxRec, frames[-1])   

shapedef = type_shapes[0]

In [ ]:
# get nearest-neighbor distance, rescaled to go from 0-1, as cval
fbox = freud.box.Box.from_box(box)
nn = freud.locality.NearestNeighbors(1.5, 1).compute(fbox, positions, positions)
cval = nn.r_sq_list[:, 0].copy()
cval -= np.min(cval)
cval /= np.max(cval)

colors = plato.cmap.cubehelix(.25 + .5*cval, s=1)
prim = draw.ConvexSpheropolyhedra(
    positions=positions, orientations=orientations, colors=colors, 
    vertices=shapedef['vertices'], radius=shapedef['rounding_radius'])
features = dict(ambient_light=.4, directional_light=directional_lights)
scene = draw.Scene(prim, features=features, zoom=1.3)

target = '../../gallery/flowws_cubes_povray.png'
scene.save(target)
IPython.display.Image(filename=target)